In [2]:
import pandas as pd
import numpy as np

# Feature Selection

### Load in the data

In [3]:
data = pd.read_csv('dataset/spotify_songs_restructured_cleaned.csv')
data.head()

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748,0.916,6.000000,-6.711124,0.058300,0.0653,0.518,122.036,194754.0,High Popularity,Fast,Moderate,June
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815,5.373157,-4.969000,0.037300,0.3570,0.693,99.972,162600.0,High Popularity,Moderate,Loud,December
2,All the Time - Don Diablo Remix,Zara Larsson,70.0,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931,1.000000,-3.432000,0.107319,0.1100,0.613,124.008,176616.0,High Popularity,Fast,Loud,July
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,pop,0.718,0.930,7.000000,-3.778000,0.102000,0.2040,0.277,121.956,169093.0,Medium Popularity,Fast,Loud,July
4,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.0,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,pop,0.675,0.919,8.000000,-5.385000,0.127000,0.1430,0.585,124.982,163049.0,High Popularity,Fast,Moderate,July


In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest

X = data[['danceability', 'energy', 'tempo', 'loudness']]  # Feature set
y = data['popularity_category']  # Target

# Apply the F-test
selector = SelectKBest(f_classif, k='all')
X_new = selector.fit_transform(X, y)

# Get the scores
f_scores = selector.scores_